# Training text models with fastai
- this notebook contains the steps to train a deep learning model on a text dataset
- the first section

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from fastai.text.all import *

# Training a language model
- take a pretrained model and train it some more using the IMDB dataset

In [ ]:
# chapter 10 language model definition
'''
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()
    
learn.fit_one_cycle(1, 2e-2)

learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

learn.save_encoder('finetuned')

'''

In [3]:
%%time
# create dataloaders object
'''dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', bs=16)
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)'''
# LSTM have multiple dropout probabilities for different things. Once you set them, this drop_mult property scales all of them. So you can change all dropout probabilities simultaneously using this, keeping their relative size
path = untar_data(URLs.IMDB)
path.ls()

CPU times: user 4.6 ms, sys: 4 µs, total: 4.6 ms
Wall time: 14.4 ms


(#7) [Path('/storage/data/imdb/README'),Path('/storage/data/imdb/tmp_lm'),Path('/storage/data/imdb/imdb.vocab'),Path('/storage/data/imdb/tmp_clas'),Path('/storage/data/imdb/test'),Path('/storage/data/imdb/train'),Path('/storage/data/imdb/unsup')]

In [4]:
# dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', bs=8)
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
dls = TextDataLoaders.from_folder(path, valid = 'test', is_lm=True, bs=16)

In [6]:
dls.show_batch(max_n=4)

,text,text_
0,"xxbos xxmaj proof why xxmaj hollywood conventions are in place . xxmaj stale dialogue , underdeveloped and flat characters and a disjointed storyline are only part of the problems with this gangster classic wannabe . xxmaj an attempt to be daring and different but this appears to be a slap - together attempt at recreating the magic of xxmaj arthur xxmaj penn 's xxmaj bonnie and xxmaj clyde ( 1967 ) and","xxmaj proof why xxmaj hollywood conventions are in place . xxmaj stale dialogue , underdeveloped and flat characters and a disjointed storyline are only part of the problems with this gangster classic wannabe . xxmaj an attempt to be daring and different but this appears to be a slap - together attempt at recreating the magic of xxmaj arthur xxmaj penn 's xxmaj bonnie and xxmaj clyde ( 1967 ) and xxmaj"
1,"no place being on any rack anywhere ; it is simply not a movie in any sense of the word . xxmaj offensive , irresponsible junk such as this can only be detrimental to the efforts being made to promote and support gay cinema ( hell , gay rights in general ! ) xxmaj for those tempting to rent this out because of the cover , you xxup will be disappointed .","place being on any rack anywhere ; it is simply not a movie in any sense of the word . xxmaj offensive , irresponsible junk such as this can only be detrimental to the efforts being made to promote and support gay cinema ( hell , gay rights in general ! ) xxmaj for those tempting to rent this out because of the cover , you xxup will be disappointed . xxmaj"
2,"loosened up , but if not , then maybe he should let others direct his words . xxmaj he 's far too precious with them here and as a result , they lose their rhythmic , jazzy quality . xxmaj what 's more strange is that other than this , the film does n't look or feel like a play . xxmaj the camera is very cinematic . xxmaj my only problem","up , but if not , then maybe he should let others direct his words . xxmaj he 's far too precious with them here and as a result , they lose their rhythmic , jazzy quality . xxmaj what 's more strange is that other than this , the film does n't look or feel like a play . xxmaj the camera is very cinematic . xxmaj my only problem with"
3,", but they just did n't seem to fit in the story . xxmaj the only link they have to the main story is that xxmaj cagney had to put on 3 previews in 3 days to get a contract and that 's what he did . i had a hard time believing that this was what the girls had been rehearsing during the entire movie and that these sets could fit",but they just did n't seem to fit in the story . xxmaj the only link they have to the main story is that xxmaj cagney had to put on 3 previews in 3 days to get a contract and that 's what he did . i had a hard time believing that this was what the girls had been rehearsing during the entire movie and that these sets could fit in


In [5]:
%%time
# define and train model
learn = language_model_learner(dls,AWD_LSTM,drop_mult=0.5,metrics=accuracy).to_fp16()
learn.fine_tune(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.517407,4.295784,0.272043,34:54


epoch,train_loss,valid_loss,accuracy,time
0,4.070133,3.992280,0.301846,32:25


CPU times: user 48min 46s, sys: 13min 44s, total: 1h 2min 30s
Wall time: 1h 7min 30s


In [9]:
# get prediction
# preds = learn.predict('The star is', n_words=20)
learn.predict("what comes next", n_words=20)

'what comes next is well worth . i saw this as a first believe and even though it was really good , it'

In [13]:
learn.export('/notebooks/temp/models/lm_model_mar2')

In [ ]:
keep_path = learn.path

In [27]:
# workaround to make path writeable
learn.path = Path('/notebooks/temp')

In [28]:
learn.path

Path('/notebooks/temp')

In [26]:
learn.model_dir

'models'

In [ ]:
learn.save('lm_mar2')

In [29]:
# workaround to save encoder - need to do this to later load encoder for classifier
learn.save_encoder('ft_mar2')

In [ ]:
learn.path = keep_path

In [19]:
learn2 = load_learner('/notebooks/temp/models/lm_model_mar2')

In [20]:
learn2.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,nan,None,None,00:00
1,nan,None,None,00:00
2,nan,None,None,00:00
3,nan,None,None,00:00
4,nan,None,None,00:00
5,nan,None,None,00:00
6,nan,None,None,00:00
7,nan,None,None,00:00
8,nan,None,None,00:00
9,nan,None,None,00:00


In [ ]:
# up to this point everything worked as expected

# classifier experiment

In [6]:
'''
%%time


dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', bs=16)
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(1, 1e-2)
'''

epoch,train_loss,valid_loss,accuracy,time
0,0.608482,0.415748,0.807680,13:54


epoch,train_loss,valid_loss,accuracy,time
0,0.305750,0.222212,0.912720,20:19


CPU times: user 24min 30s, sys: 9min 51s, total: 34min 22s
Wall time: 34min 20s


In [22]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [31]:
path

Path('/storage/data/imdb')

In [23]:
learn_clas = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [32]:
# learn.path = Path('/notebooks/temp')
learn_clas.path = Path('/notebooks/temp')

In [33]:
learn_clas = learn_clas.load_encoder('ft_feb28')

In [34]:
learn_clas.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.416318,0.279885,0.884360,03:10


In [36]:
%%time
learn_clas.freeze_to(-2)
learn_clas.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.258281,0.202682,0.921920,03:32


In [37]:
preds = learn_clas.predict("this film shows incredibly bad writing and is a complete disaster")

In [40]:
preds

('pos', TensorText(1), TensorText([0.0032, 0.9968]))

In [39]:
preds = learn_clas.predict("what a wonderful film")

In [8]:
learn_clas.save('classifier_single_epoch_mar2')

OSError: [Errno 30] Read-only file system: '/storage/data/imdb/models'